# 국가별 수산물 수출입 현황

In [ ]:
import requests, json
import pandas as pd
import numpy as np
from pprint import pprint
from tqdm.notebook import tqdm

In [ ]:
URL = 'https://apis.data.go.kr/1192000/select0090List/getselect0090List'
API_KEY = 'a46bbaf106e41963c3883db630366f91e4960f1a08c831475517864811d806f4'

In [ ]:
query = {
    'serviceKey': API_KEY,
    'numOfRows': 100,
    'pageNo': 1,
    'type': 'json',
    'baseDt': '202511',
    'nationNm': '일본'
}

In [ ]:
res = requests.get(url=URL, params=query)

In [ ]:
res.status_code

In [ ]:
res.headers['content-type']

In [ ]:
res.text

In [ ]:
# json 문자열 -> 딕셔너리 변환
data = json.loads(s = res.text)

In [ ]:
pprint(data)

In [ ]:
tcnt = data['responseJson']['header']['totalCount']
print(tcnt)

pages = int(np.ceil(tcnt / 100))
print(pages)

In [ ]:
for page in range(pages):
    page += 1

In [ ]:
df = pd.DataFrame(data=data['responseJson']['body']['item'])
df.head()

### 함수로 만들기

In [ ]:
def marine_product(ymonth, nation=None, nrow=100, page=1):
    URL = 'https://apis.data.go.kr/1192000/select0090List/getselect0090List'
    API_KEY = 'a46bbaf106e41963c3883db630366f91e4960f1a08c831475517864811d806f4'
    query = {
        'serviceKey': API_KEY,
        'numOfRows': nrow,
        'pageNo': page,
        'type': 'json',
        'baseDt': ymonth,
        'nationNm': nation
    }
    res = requests.get(url=URL, params=query)
    data = json.loads(s = res.text)
    df = pd.DataFrame(data=data['responseJson']['body']['item'])
    return df

In [ ]:
df1 = marine_product(ymonth='202511', nation='일본', page=1)
df1.shape

In [ ]:
df2 = marine_product(ymonth='202511', nation='일본', page=2)
df2.shape

In [ ]:
df1.equals(df2)
# False

In [ ]:
df2.equals(df2)
# True

In [ ]:
dfs = pd.DataFrame()
page = 0
while True:
    page += 1
    df = marine_product(ymonth='202511', nation='일본', page=page)
    tails = dfs.tail(100).reset_index(drop=True)
    nrows = df.shape[0]
    if df.equals(tails):
        break
    elif nrows < 100:
        dfs = pd.concat([dfs, df], ignore_index=True)
        break
    else:
        dfs = pd.concat([dfs, df], ignore_index=True)

In [ ]:
dfs.shape

In [ ]:
def marine_product_monthly(ymonth, nation):
    dfs = pd.DataFrame()
    page = 0
    while True:
        page += 1
        df = marine_product(ymonth=ymonth, nation=nation, page=page)
        tails = dfs.tail(100).reset_index(drop=True)
        nrows = df.shape[0]
        if df.equals(tails):
            break
        elif nrows < 100:
            dfs = pd.concat([dfs, df], ignore_index=True)
            break
        else:
            dfs = pd.concat([dfs, df], ignore_index=True)
    return dfs

In [ ]:
df = marine_product_monthly(ymonth='202511', nation='일본')
df.shape